In [12]:
import requests
import hashlib
import time
import pandas as pd
from pandas import json_normalize

secret_token = '696a6f47212d93f88cef71238cfb3d1cd357905b3e34eabd2bbc49124553c129'

# Generate the Unix timestamp
timestamp = int(time.time())

# Construct the query string
query_string = f'time={timestamp}'

# Generate the hash
hash_string = secret_token + str(timestamp)
hash_value = hashlib.sha512(hash_string.encode()).hexdigest()

# Construct the URL with the query string and hash
url = f'https://api.fabdb.net/cards?{query_string}&hash={hash_value}'

parameters = {
    "page": 1,
    "per_page": 100
}

total_pages = 21

# Set the request headers
headers = {
    'Authorization': f'Bearer {secret_token}',
    'Accept': 'application/json'
}


In [13]:
#empty list to hold paginated data
data_records = []

for page in range(1, total_pages + 1):
    parameters['page'] = page
    
    response = requests.get(url, params=parameters)

    # process response
    data = response.json()
    
    # Extract the relevant data records from the response
    data_records.extend(data['data'])

# Normalize the data before storing it in the dataframe
df = pd.json_normalize(data_records, max_level=3)
df_exploded = df.explode('printings')

# Create separate columns for 'id', 'language', and 'name'
df_exploded[['id', 'language', 'name', 'text', 'flavour', 'sku']] = pd.DataFrame(df_exploded['printings'].tolist())

# Drop the original 'printings' column
df_exploded = df_exploded.drop('printings', axis=1)

# Output the exploded DataFrame
print(df_exploded)

ValueError: Columns must be same length as key

In [ ]:
df.rename(columns={'stats.resource': 'pitch value'}, inplace=True)

# Map the values in 'pitch value' column to corresponding colors
df['pitch value'] = df['pitch value'].map({1: 'Red', 2: 'Yellow', 3: 'Blue'})

# Rename other columns
df.rename(columns={'stats.life': 'life', 'stats.intellect': 'intellect','stats.attack': 'attack', 'stats.defense': 'defense', 'stats.cost': 'cost'}, inplace=True)
#drop stats column after it's been split
columns_to_drop = ['stats']
df.drop(columns_to_drop, axis=1, inplace=True)
df.head(2)

,identifier,name,legality,keywords,text,rarity,image,printings,pitch value,life,intellect,attack,defense,cost
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 863, 'language': 'en', 'name': 'Eye of...",Blue,NaN,NaN,NaN,NaN,NaN
1,dash-inventor-extraordinaire,"Dash, Inventor Extraordinaire",[],"[mechanologist, hero]",You may start the game with a Mechanologist it...,T,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 864, 'language': 'en', 'name': 'Dash, ...",NaN,40.0,4.0,NaN,NaN,NaN


In [ ]:
#convertng selected columns to numbers for statistical analysis
columns_to_convert = ['life', 'attack', 'defense', 'cost', 'intellect']
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce', downcast='float')
pd.set_option('display.max_rows', None)  # Show all rows
df.head(2)

,identifier,name,legality,keywords,text,rarity,image,printings,pitch value,life,intellect,attack,defense,cost
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 863, 'language': 'en', 'name': 'Eye of...",Blue,NaN,NaN,NaN,NaN,NaN
1,dash-inventor-extraordinaire,"Dash, Inventor Extraordinaire",[],"[mechanologist, hero]",You may start the game with a Mechanologist it...,T,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 864, 'language': 'en', 'name': 'Dash, ...",NaN,40.0,4.0,NaN,NaN,NaN
